Private score 0.96, Public 0.99325.

Хотел выложить свой результат, мало ли кто поделится хорошими идеями, а кому-то пригодится. 
Взял 11 Efficient_b0, каждую из них обучил на отдельной рандомной выборке из тренеровочных данных.
Пытался таким образом избежать переобучния. Результаты сеток складываются. И делается предикшн.
Сетки тренеруются в 2 этапа, вначале только классификатор, сама сеть заморожена(30 эпох), потом дообучается полностью размороженая(100 эпох).
Так как в данном случае мы не боимся переобучаться, то dropout убран из классификаторая, для более быстрой сходимости.
Если есть вопросы задавайте, если есть хорошие советы, обязательно говорите =))


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

import torch
from torchvision.models import efficientnet_b0 as eff
from torch.utils.data import Dataset
from torchvision import datasets, transforms
import torch.optim as optim


from sklearn.metrics import accuracy_score, confusion_matrix
from torch.utils.tensorboard import SummaryWriter

e:\pythonprojects\python3.9.5_wood\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

Using cuda for inference


In [3]:
eff_net = eff(pretrained=True)

In [4]:
train_path = 'data2/'
test_path = 'data2/'
val_path = 'test2/'

In [5]:
def ohe(lable: str) -> list[3]:
    """One hot encode lable.

    Args:
        lable (str): '0' or '2' or '3'.

    Returns:
        list[3]: list len 3 with 1 in position of lable(0->0, 2->1, 3->2).
    """
    temp = [0,0]
    temp.insert(int(lable),1)
    return temp
    
def lable_encode_to_ohe(tensor: torch.tensor) -> torch.tensor:
    """ohe lables in tensor

    Args:
        tensor (torch.tensor): tensor with lables.

    Returns:
        torch.tensor: tensor with ohe-ed lables.
    """
    return torch.tensor([ohe(lable) for lable in tensor], dtype=float).to(device)

In [6]:
img_size = (3,224,224)
freeze_batch_size = 128
unfreeze_batch_size = 10


In [7]:
class WoodImageDataset(Dataset):
    """
    Creates dataset, used for creating result for competition.
    
    Returns:
        img : img np.array[h,w].
        lable: int.
    """
    
    def  __init__(self, img_folder, transform=None):
        
        self.img_folder = img_folder
        self.transform=transform
        self.all_files = os.listdir(img_folder)

    def __len__(self):
        return len(self.all_files)

    def __getitem__(self,idx):

        img = Image.open(self.img_folder + str(idx + 1) +'.png')
        label = idx + 1

        if self.transform:
            img = self.transform(img)

        return img , label

In [8]:
class DatasetForBagging(Dataset):
    """
    Creates dataset which randomly takes values from training data. Created
    for bagging.
    
    self.index_for_val - contains indexes which didn't enter in created dataset. Used for test.
    """
    def __init__(self, folder_path, transform=None, idxs_list=None):
        self.transform = transform
        self.folder_path = folder_path
        
        self.class_names = os.listdir(self.folder_path)
        self.labels = [i for i in range(len(self.class_names))]
        self.labels_to_class = dict(zip(self.labels, self.class_names))
        self.class_to_labels = dict(zip(self.class_names, self.labels))

        self.file_paths, self.file_labels = self.get_file_list(self.folder_path)


        if not idxs_list:
            self.random_idxs = [np.random.randint(0, len(self.file_labels)) for _ in range(len(self.file_labels))]
            self.all_idxs = set(range(len(self.file_labels)))
            self.idxs_for_val = list(set.difference(self.all_idxs, (set(self.random_idxs))))
        else:
            self.random_idxs = idxs_list
        

    def get_file_list(self, img_folder_path):

        file_paths = []
        file_labels = []
        
        for class_name in self.class_names:
            for file_name in os.listdir(img_folder_path + class_name):
                file_paths.append(img_folder_path + class_name + '/' + file_name)
                file_labels.append(int(self.class_to_labels[class_name]))
        return file_paths, file_labels

    def __len__(self):
        return len(self.random_idxs)
    
    def __getitem__(self, idx):
        index = self.random_idxs[idx]
        img = Image.open(self.file_paths[index])
        label = self.file_labels[index]

        if self.transform:
            img = self.transform(img)

        return img, label


In [9]:

train_transform = transforms.Compose([
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5),
    transforms.Resize(img_size[1:]),
    transforms.ToTensor(),
    # CircleTranform(circle_filter.pixel_filter.astype('float32')),
    transforms.ColorJitter(0.15,0.15)
    ]
)

test_transform = transforms.Compose([
    # transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5),
    transforms.Resize(img_size[1:]),
    transforms.ToTensor(),
    # CircleTranform(circle_filter.pixel_filter.astype('float32')),
    transforms.ColorJitter(0.1,0.1)
    ]
)

val_transform = transforms.Compose([
    transforms.Resize(img_size[1:]),
    transforms.ToTensor(),
    # CircleTranform(circle_filter.pixel_filter.astype('float32'))
    ]
)

In [11]:
num_of_models = 11

In [12]:
train_datasets = [DatasetForBagging(train_path, transform=train_transform) for _ in range(num_of_models)]
test_datasets = [DatasetForBagging(train_path, transform=val_transform, idxs_list = train_datasets[i].idxs_for_val) for i in range(num_of_models)]
val_dataset = WoodImageDataset(val_path, transform=val_transform)

In [14]:
train_loaders = [torch.utils.data.DataLoader(train_datasets[i], batch_size=freeze_batch_size, shuffle=True, drop_last=True) for i in range(num_of_models)]
test_loaders = [torch.utils.data.DataLoader(test_datasets[i], batch_size=freeze_batch_size, shuffle=True, drop_last=True) for i in range(num_of_models)]
train_loaders = [torch.utils.data.DataLoader(train_datasets[i], batch_size=16, shuffle=True, drop_last=True) for i in range(num_of_models)]
test_loaders = [torch.utils.data.DataLoader(test_datasets[i], batch_size=16, shuffle=True, drop_last=True) for i in range(num_of_models)]

val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32)

In [15]:
# for data in train_loader1:
#     imgs, labels = data
#     print(np.unique(labels))

In [16]:

class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.eff = torch.nn.Sequential(
            eff_net.features,
            eff_net.avgpool,
            torch.nn.Flatten()
        )
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(in_features=1280, out_features=3, bias=True),
            torch.nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.eff(x)
        x = self.classifier(x)
        return x


nets = [Net().to(device) for _ in range(num_of_models)]

In [18]:
def train(model, dataloader, num_epoch, freeze_backbone=True):
    epoch_loss = []
    if freeze_backbone:
        for param in model.eff.parameters():
            param.requires_grad = False
    else:
        for param in model.eff.parameters():
            param.requires_grad = True
        
    for batch in dataloader:
        inputs, labels = batch
        inputs = inputs.to(device)

        optimizer.zero_grad()
        outputs = model(inputs) # already softmaxed

        loss = criterion(outputs, lable_encode_to_ohe(labels))
        epoch_loss.append(loss.tolist())
        loss.backward()
        optimizer.step()

    writer.add_scalar('loss/train', np.mean(epoch_loss), num_epoch)
    num_epoch += 1
    return num_epoch


In [19]:
def val(model, dataloader, num_epoch ,name):
    
    with torch.no_grad():

        preds = []
        true_labels = []

        for data in dataloader:
            img, labels = data
            batch_preds = model(img.to(device))
            preds.append(batch_preds)
            true_labels.append(labels)

        result = []
        for batch_preds in preds:
            result.extend([torch.argmax(i).tolist() for i in batch_preds])

        true_labels_result = []
        for true_labels_batch in true_labels:

            true_labels_result.extend(true_labels_batch)

        accuracy = accuracy_score(true_labels_result, result)
        print(f'{name} accuracy =' , accuracy)

        writer.add_scalar('accuracy/test', accuracy, num_epoch)
        
        print(confusion_matrix(true_labels_result,result))


In [20]:
criterion = torch.nn.CrossEntropyLoss()
freeze_lr = 0.01
freeze_epoch_number = 30

In [21]:
for num_of_net in range(len(nets)):
    writer = SummaryWriter('runs/wood_ensemble1/' + f'net#{num_of_net}', comment=f'net#{num_of_net}')
    optimizer = optim.AdamW(nets[num_of_net].parameters(), lr=freeze_lr)
    num_epoch = 0
    for epoch in range(freeze_epoch_number):
        print(f'epoch #{epoch}')
        num_epoch = train(nets[num_of_net], train_loaders[num_of_net], num_epoch, freeze_backbone=True)
        if epoch % 5 == 0:
            val(nets[num_of_net], test_loaders[num_of_net], num_epoch, 'test')

epoch #0
test accuracy = 0.7596153846153846
[[ 0 23 12]
 [ 0 97  8]
 [ 0  7 61]]
epoch #1
epoch #2
epoch #3
epoch #4
epoch #5
test accuracy = 0.7740384615384616
[[  0  22  12]
 [  0 100   6]
 [  0   7  61]]
epoch #6
epoch #7
epoch #8
epoch #9
epoch #10
test accuracy = 0.8557692307692307
[[ 19  13   3]
 [  3 102   0]
 [  3   8  57]]
epoch #11
epoch #12
epoch #13
epoch #14
epoch #15
test accuracy = 0.8173076923076923
[[15  6 14]
 [ 1 90 14]
 [ 0  3 65]]
epoch #16
epoch #17
epoch #18
epoch #19
epoch #20
test accuracy = 0.8509615384615384
[[ 17  13   5]
 [  2 100   4]
 [  0   7  60]]
epoch #21
epoch #22
epoch #23
epoch #24
epoch #25
test accuracy = 0.8846153846153846
[[30  3  2]
 [ 5 97  3]
 [ 4  7 57]]
epoch #26
epoch #27
epoch #28
epoch #29
epoch #0
test accuracy = 0.7788461538461539
[[  0  23   8]
 [  0 106   7]
 [  0   8  56]]
epoch #1
epoch #2
epoch #3
epoch #4
epoch #5
test accuracy = 0.8028846153846154
[[  0   6  22]
 [  0 103  12]
 [  1   0  64]]
epoch #6
epoch #7
epoch #8
epoch #9

In [22]:
unfreeze_epoch_number = 100
unfreeze_lr = 0.00001

In [23]:

for num_of_net in range(len(nets)):
    writer = SummaryWriter('runs/wood_ensemble/' + f'net#{num_of_net}', comment=f'net#{num_of_net}')
    optimizer = optim.AdamW(nets[num_of_net].parameters(), lr=unfreeze_lr)
    num_epoch = freeze_epoch_number
    for epoch in range(unfreeze_epoch_number):  # loop over the dataset multiple times
        print(f'epoch #{epoch}')
        num_epoch = train(nets[num_of_net], train_loaders[num_of_net], num_epoch, freeze_backbone=False)
        if epoch % 5 == 0:
            val(nets[num_of_net], test_loaders[num_of_net], num_epoch, 'test')

epoch #0
test accuracy = 0.8557692307692307
[[25  5  5]
 [ 3 88 14]
 [ 0  3 65]]
epoch #1
epoch #2
epoch #3
epoch #4
epoch #5
test accuracy = 0.9038461538461539
[[26  2  7]
 [ 0 98  8]
 [ 1  2 64]]
epoch #6
epoch #7
epoch #8
epoch #9
epoch #10
test accuracy = 0.8942307692307693
[[27  5  3]
 [ 2 96  7]
 [ 1  4 63]]
epoch #11
epoch #12
epoch #13
epoch #14
epoch #15
test accuracy = 0.9182692307692307
[[ 26   4   4]
 [  0 100   6]
 [  0   3  65]]
epoch #16
epoch #17
epoch #18
epoch #19
epoch #20
test accuracy = 0.9038461538461539
[[27  4  4]
 [ 2 98  6]
 [ 2  2 63]]
epoch #21
epoch #22
epoch #23
epoch #24
epoch #25
test accuracy = 0.9038461538461539
[[23  2 10]
 [ 1 98  6]
 [ 1  0 67]]
epoch #26
epoch #27
epoch #28
epoch #29
epoch #30
test accuracy = 0.9086538461538461
[[27  4  4]
 [ 1 97  7]
 [ 0  3 65]]
epoch #31
epoch #32
epoch #33
epoch #34
epoch #35
test accuracy = 0.9038461538461539
[[27  3  5]
 [ 1 97  8]
 [ 2  1 64]]
epoch #36
epoch #37
epoch #38
epoch #39
epoch #40
test accuracy =

In [24]:
def bagging_prediction(models, dataloader, save_path=''):
    
    with torch.no_grad():
        models_predictions = []

        for model in models:

            preds = []
            file_ids = []

            for data in dataloader:
                img, ids = data
                batch_preds = model(img.to(device))
                preds.extend(batch_preds.tolist())
                file_ids.extend(ids.tolist())

            models_predictions.append(preds)

        bagging_result = np.array(models_predictions)

        
            
    return file_ids, np.argmax(np.mean(bagging_result,axis=0), axis=1)

In [25]:
preds = bagging_prediction(nets, val_loader)

In [26]:
df = pd.DataFrame(preds).T
df.columns = ('id', 'class')
df['class'] = df['class'].apply(lambda x: 3 if x==2 else x)
df.to_csv('hello.csv', index=0)